In [78]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import pandas as pd
import time


In [2]:
kospi_df = pd.read_csv('kospi_company_info.csv')
kospi_codes = kospi_df['stock_code']

In [9]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)

In [10]:
code = '271980'

In [24]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 15)
driver.get('https://kind.krx.co.kr/disclosure/searchdisclosurebycorp.do?method=searchDisclosureByCorpMain')

original_window = driver.current_window_handle

In [25]:
report_element = wait.until(EC.presence_of_element_located((By.ID, 'AKCKwd')))
report_element.clear()
report_element.send_keys(code)

# input the start date
date_element = wait.until(EC.element_to_be_clickable((By.ID, 'fromDate')))
date_element.clear()
date_element.send_keys('2025-01-01')

# input the target document title (Disclosure of Corporate Governance Report)
report_element = wait.until(EC.presence_of_element_located((By.ID, 'reportNmTemp')))
report_element.send_keys('기업지배구조 보고서 공시')

# click on the search button
search_element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.btn-sprite.search-btn')))
search_element.click()

# wait for the search results to load
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'tbody tr')))

# extract the table from the first search result, which will be the most recent report
disclosure_link = wait.until(EC.element_to_be_clickable(((By.XPATH, "//a[contains(text(), '기업지배구조 보고서 공시')]"))))
disclosure_link.click()

In [26]:
all_window_handles = driver.window_handles
for handle in all_window_handles:
    if handle != original_window:
        driver.switch_to.window(handle)
        break
print(f'New window: {driver.title}')

New window: [제일약품] [정정]기업지배구조 보고서 공시


In [27]:
iframe = wait.until(EC.presence_of_element_located((By.ID, "docViewFrm")))
driver.switch_to.frame(iframe)

In [ ]:
css_selector = 'td.single-textbox.bg_percent'

try:
    element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, css_selector)))
    report_value = element.text

except Exception as e:
    print(f"Error: Element not found or not present. {e}")

The extracted text is: 26.7
The value from the 'value' attribute is: 26.7


In [29]:
try:
    css_selector = 'table-group[aclass="krx-cg_VotingResultsOfTheGeneralMeetingOfShareholdersAbstract"] table.fact-table'
    fact_table_element = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, css_selector))
    )

    table_html_string = fact_table_element.get_attribute('outerHTML')

    # Use BeautifulSoup to parse for headers.
    soup = BeautifulSoup(table_html_string, 'html.parser')
    scraped_headers = [th.get_text(strip=True) for th in soup.find_all('th')]

    # Use pandas to read the table from the HTML string.
    dfs = pd.read_html(table_html_string, header=None)

    if dfs:
        df = dfs[0]

        # Define final headers with the first two columns.
        final_headers = ['총회', '의안'] + scraped_headers[1:]
        
        # Clean the DataFrame to match the number of headers.
        df = df.iloc[1:]
        df.reset_index(drop=True, inplace=True)
        
        # Rename columns with the new headers.
        if len(final_headers) == len(df.columns):
            df.columns = final_headers
            print(f"Successfully extracted and processed table.")
        else:
            print(f"Error: The number of columns ({len(df.columns)}) does not match the number of headers ({len(final_headers)}).")
    else:
        print("No tables found in the HTML.")
        
except Exception as e:
    print(f"Error extracting table.")
    
finally:
    # Switch back to the main document from the iframe, regardless of success or failure.
    driver.switch_to.default_content()

Successfully extracted and processed table.


C:\Users\Owner\AppData\Local\Temp\ipykernel_2576\897760325.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(table_html_string, header=None)


In [30]:
df

,총회,의안,결의 구분,회의 목적사항,가결 여부,의결권 있는 발행주식 총수(1),(1) 중 의결권 행사 주식수,찬성주식수,찬성 주식 비율 (%),반대 기권 등 주식수,반대 기권 등 주식 비율 (%)
0,제 8기 정기 주주총회,제2호 의안,보통(Ordinary),사내이사 선임의 건 사내이사 : 노치국(재선임) 사내이사 : 이창석(재선임) ...,가결(Approved),14704872,9332838,9332838,100.0,0,0.0
1,제 8기 정기 주주총회,제3호 의안,보통(Ordinary),사외이사 선임의 건 사외의사 : 김득웅(신규선임),가결(Approved),14704872,9332838,9332838,100.0,0,0.0
2,제 8기 정기 주주총회,제4호 의안,보통(Ordinary),사외이사인 감사위원 선임의 건 사외이사인 감사위원 : 김득웅(신규선임),가결(Approved),14704872,9332838,9332838,100.0,0,0.0
3,제 8기 정기 주주총회,제5호 의안,보통(Ordinary),이사 보수한도 승인의 건,가결(Approved),14704872,9332838,9332838,100.0,0,0.0
4,제 7기 정기 주주총회,제1호 의안,보통(Ordinary),제7기(2023.1.1 ~ 2023.12.31 ) 재무제표(이익잉여금처분계산서) ...,가결(Approved),14704872,11484450,11472512,99.9,11938,0.1
5,제 7기 정기 주주총회,제2호 의안,보통(Ordinary),이사 보수한도 승인의 건,가결(Approved),14704872,11484450,11216957,97.7,267493,2.3
6,제 6기 정기 주주총회,제1호 의안,보통(Ordinary),제6기(2022.1.1~2022.12.31) 재무제표 (이익잉여금처분계산서포함) ...,가결(Approved),14704872,11486426,11471578,99.9,14848,0.1
7,제 6기 정기 주주총회,제2호 의안,보통(Ordinary),제2호 의안 : 정관 일부 변경의 건,가결(Approved),14704872,11486426,11485797,100.0,629,0.0
8,제 6기 정기 주주총회,제3호 의안,보통(Ordinary),제3호 의안 :사내이사 선임의 건 3-1호 사내이사 : 성석제 (재선임) 3-2...,가결(Approved),14704872,11486426,11429008,99.5,57418,0.5
9,제 6기 정기 주주총회,제4호 의안,보통(Ordinary),제4호 의안 : 사외이사 선임의 건 4-1호 사외이사 : 정병도 (재선임) 4-...,가결(Approved),14704872,11486426,11168119,97.2,318307,2.8


In [ ]:
def scrape_governance_data(driver, codes_list):
    results_dict = {}
    wait = WebDriverWait(driver, 1)
    
    for code in codes_list:
        driver.get('https://kind.krx.co.kr/disclosure/searchdisclosurebycorp.do?method=searchDisclosureByCorpMain')
        original_window = driver.current_window_handle
        try:            
            # Use JavaScript to input the code
            report_element = wait.until(EC.presence_of_element_located((By.ID, 'AKCKwd')))
            driver.execute_script("arguments[0].value = arguments[1];", report_element, code)
            
            # Input the start date
            time.sleep(.2)

            date_element = wait.until(EC.element_to_be_clickable((By.ID, 'fromDate')))
            date_element.clear()
            date_element.send_keys('2025-01-01')
                        
            time.sleep(.2)

            # Input the document title
            report_element = wait.until(EC.presence_of_element_located((By.ID, 'reportNmTemp')))
            report_element.send_keys('기업지배구조 보고서 공시')
    
            # Click the search button
            search_element = driver.find_element(By.CSS_SELECTOR, 'a.btn-sprite.search-btn')
            search_element.click()
            
            time.sleep(.2)
            try:
                disclosure_link = driver.find_element(By.XPATH, "//tbody/tr[1]//a[contains(text(), '기업지배구조 보고서 공시')]")
                disclosure_link.click()
            except Exception as e:
                print(f"No search results or link found for code {code}. Skipping...")
                continue 
            # Wait for a new window to open and switch to it
            wait.until(EC.number_of_windows_to_be(2))
            all_window_handles = driver.window_handles
            new_window_handle = [handle for handle in all_window_handles if handle != original_window]

            if new_window_handle:
                driver.switch_to.window(new_window_handle[0])
                print(f'Processing code {code}, new window: {driver.title}')
                
                # Switch to the iframe
                iframe = wait.until(EC.presence_of_element_located((By.ID, "docViewFrm")))
                driver.switch_to.frame(iframe)

                # Extract the report value and table
                report_value = None
                df = None
                
                try:
                    report_value_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'td.single-textbox.bg_percent')))
                    report_value = report_value_element.text
                except Exception as e:
                    print(f"Warning: Could not find report value for code {code}. Error: {e}")

                try:
                    css_selector = 'table-group[aclass="krx-cg_VotingResultsOfTheGeneralMeetingOfShareholdersAbstract"] table.fact-table'
                    fact_table_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, css_selector)))
                    table_html_string = fact_table_element.get_attribute('outerHTML')
                    
                    soup = BeautifulSoup(table_html_string, 'html.parser')
                    scraped_headers = [th.get_text(strip=True) for th in soup.find_all('th')]
                    
                    dfs = pd.read_html(table_html_string, header=None)
                    if dfs:
                        df = dfs[0]
                        final_headers = ['총회', '의안'] + scraped_headers[1:]
                        df = df.iloc[1:]
                        df.reset_index(drop=True, inplace=True)
                        
                        if len(final_headers) == len(df.columns):
                            df.columns = final_headers
                            print(f"Successfully extracted table for code {code}.")
                        else:
                            print(f"Column/header mismatch for code {code}.")
                            df = None
                    else:
                        print(f"No tables found for code {code}.")
                except Exception as e:
                    print(f"Error extracting table for code {code}. Error: {e}")
                    df = None
                
                results_dict[code] = {'report_value': report_value, 'df': df}
                
                # Close the new window and switch back to the original
                driver.close()
                driver.switch_to.window(original_window)
            else:
                print(f"No new window opened for code {code}.")
        
        except Exception as e:
            alert = driver.switch_to.alert
            print(f"Alert on page for code {code}. Text: {alert.text}")
            alert.accept()
            print("Alert dismissed. Skipping to next code.")
            # Note: The loop will continue, and `driver.get()` will be called to reset the state.
        except Exception as e:
            print(f"An error occurred while processing code {code}: {e}")
            
    return results_dict

if __name__ == '__main__':
    driver = webdriver.Chrome()
    stock_codes_to_scrape = kospi_codes
    governance_data = scrape_governance_data(driver, stock_codes_to_scrape)
    
    for code, data in governance_data.items():
        print(f"\nData for code: {code}")
        print(f"  Report Value: {data['report_value']}")
        print("  DataFrame:")
        if data['df'] is not None:
            data['df'].head()
        else:
            print("No DataFrame available.")
    
    driver.quit()

In [11]:
ranked_voting = df[df['반대 기권 등 주식수'].isna()]
majority_voting = df[df['반대 기권 등 주식수'].notna()]

In [12]:
ranked_voting

,총회,의안,결의 구분,회의 목적사항,가결 여부,의결권 있는 발행주식 총수(1),(1) 중 의결권 행사 주식수,찬성주식수,찬성 주식 비율 (%),반대 기권 등 주식수,반대 기권 등 주식 비율 (%)
16,제37기 정기 주주총회,제3-1호 의안,보통(Ordinary),대표이사 사장 방경만 선임의 건 (KT&G 이사회 안),가결(Approved),112809923,165207264,84097688,50.9,NaN,0.0
17,제37기 정기 주주총회,제3-2호 의안,보통(Ordinary),사외이사 임민규 선임의 건 (KT&G 이사회 안),부결(Not approved),112809923,165207264,24505618,14.8,NaN,0.0
18,제37기 정기 주주총회,제3-3호 의안,보통(Ordinary),사외이사 손동환 선임의 건 (주주제안_중소기업은행),가결(Approved),112809923,165207264,56603958,34.3,NaN,0.0


In [13]:
majority_voting

,총회,의안,결의 구분,회의 목적사항,가결 여부,의결권 있는 발행주식 총수(1),(1) 중 의결권 행사 주식수,찬성주식수,찬성 주식 비율 (%),반대 기권 등 주식수,반대 기권 등 주식 비율 (%)
0,제38기 정기 주주총회,제2-1호 의안,특별(Extraordinary),이사의 인원수 명확화,가결(Approved),107856043,91736706,91628348,99.9,108358.0,0.1
1,제38기 정기 주주총회,제2-2호 의안,특별(Extraordinary),감사위원 선임 관련 조문 정비,가결(Approved),107856043,91736706,77909093,84.9,13827613.0,15.1
2,제38기 정기 주주총회,제2-3호 의안,특별(Extraordinary),대표이사 사장 선임 방법 명확화,가결(Approved),107856043,74689281,53946867,72.2,20742414.0,27.8
3,제38기 정기 주주총회,제2-4호 의안,특별(Extraordinary),분기배당기준일 변경,가결(Approved),107856043,91736706,91626520,99.9,110186.0,0.1
4,제38기 정기 주주총회,제3호 의안,보통(Ordinary),사내이사 이상학 선임의 건,가결(Approved),107856043,91736706,90539809,98.7,1196897.0,1.3
5,제38기 정기 주주총회,제4-1호 의안,보통(Ordinary),사외이사 손관수 선임의 건,가결(Approved),107856043,91736706,89269740,97.3,2466966.0,2.7
6,제38기 정기 주주총회,제4-2호 의안,보통(Ordinary),사외이사 이지희 선임의 건,가결(Approved),107856043,91736706,90424942,98.6,1311764.0,1.4
7,제38기 정기 주주총회,제5호 의안,보통(Ordinary),감사위원회 위원 손관수 선임의 건,가결(Approved),107856043,74689281,73254834,98.1,1434447.0,1.9
8,제38기 정기 주주총회,제6호 의안,보통(Ordinary),이사 보수한도 승인의 건,가결(Approved),107856043,87807386,87469388,99.6,337998.0,0.4
9,제37기 정기 주주총회,제1호 의안,보통(Ordinary),제37기 재무제표 및 이익잉여금처분계산서 승인의 건,가결(Approved),112809923,87368552,83585189,95.7,3783363.0,4.3


In [ ]:
driver.quit()